# Import

In [15]:
import json
from tqdm import tqdm

from jovis_model.config import Config
from run import ModelRunner

### CLIP Test

In [1]:
from jovis_model.models.llm.mclip import CLIPModel

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
params = {
    "pkg": "llm",
    "task": "mclip",
    "use_hf_model": True,
    "params": {
        "hf_name": "M-CLIP/XLM-Roberta-Large-Vit-B-32"
    }
}
config = Config(**params)
model = CLIPModel(config)

In [ ]:
model, preprocess = clip.load("ViT-B/32")

100%|███████████████████████████████████████| 338M/338M [00:34<00:00, 10.3MiB/s]


In [ ]:
outputs = model.inference(["안녕하세요"])

In [ ]:
file_path = "./jovis_model/_db/llm/multimodel_test/export_data_skb.json"
with open(file_path, "r") as f:
    data = json.load(f)

In [ ]:
from PIL import Image

In [ ]:
data["693669"]["s3_url"]

'https://flush-image-prod.s3.ap-northeast-2.amazonaws.com/01H5S0JRNJEDPD3KX414FBPH2K/693669.webp'

In [ ]:
image = Image.open(data["693669"]["s3_url"]).convert("RGB")

FileNotFoundError: [Errno 2] No such file or directory: '/home/omnious/workspace/jovis/jovis-model/https:/flush-image-prod.s3.ap-northeast-2.amazonaws.com/01H5S0JRNJEDPD3KX414FBPH2K/693669.webp'

### NER

In [2]:
# params = {
#     "pkg": "llm",
#     "task": "chat",
#     "use_hf_model": True,
#     "params": {
#         "hf_name": "meta-llama/Meta-Llama-3-8B-Instruct"
#     }
# }

params = {
    "pkg": "llm",
    "task": "bedrock",
    "use_hf_model": False,
    "params": {
    }
}
config = Config(**params)
runner = ModelRunner(
    config=config,
    mode="inference"
)

In [5]:
dialogs = [
    {"role": "user", "content": "What is you name?"}
]

In [6]:
out = runner.run(
    sample_inputs=dialogs
)

### InternVL

In [4]:
params = {
    "pkg": "llm",
    "task": "internvl",
    "use_hf_model": True,
    "params": {
        "hf_name": "OpenGVLab/InternVL-Chat-V1-5",
        "max_new_tokens": 512
    }
}
config = Config(**params)
runner = ModelRunner(
    config=config,
    mode="inference"
)

### Sentence Embedding

In [2]:
params = {
    "pkg": "llm",
    "task": "sentence_embedding",
    "use_hf_model": True,
    "params": {
        "hf_name": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    }
}
config = Config(**params)
runner = ModelRunner(
    config=config,
    mode="inference"
)

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
samples = [
    "눈의여왕에서 김지원이 사용한 핑크색 가방의 디자인은 어떤 거야?",
    "요즘 방영 중인 눈의여왕 드라마에서 주인공이 입고 나온 스타일의 청바지를 찾아줘.",
    "미스터 선샤인 에서 주인공이 입은 모자의 디자인을 찾고 있어요.",
    "최근 예능 프로그램에서 나온 배우 김지원의 데님 팬츠를 찾고 있어요.",
    "지난 주 “눈의여왕” 드라마에서 주인공이 입은 외투를 찾아줄 수 있어?",
    "어느 멋진 날에서 MC가 입은 슈트의 색상을 알려주세요.",
    "최근에 방영된 “마이데몬”에서 주인공이 입은 신발의 모델명은 무엇이야?",
    "어제 방송된 “눈의여왕”에서 여주인공이 입은 원피스의 색상은?",
    "요즘 인기 있는 드라마에서 주인공이 입은 아우터를 알려줘.",
    "최근 방영된 “마이데몬”에서 주인공이 입은 액세서리가 돋보이는데, 어디서 구할 수 있어?",
    "어제 방송된 드라마에서 여주인공이 입은 셔츠의 디자인을 찾아줄래?",
    "최신 드라마에서 남자 주인공이 입은 코트를 찾아줘."
]

In [10]:

outputs = runner.run(samples[:1])

In [4]:
file_path = "/home/omnious/workspace/jovis/jovis-model/outputs/skb/descriptions_v2.json"
with open(file_path, "r") as f:
    descriptions = json.load(f)

In [17]:
pids = []
embeddings = []
for pid, description in tqdm(descriptions.items()):
    pids.append(pid)
    embeddings.append(runner.run([description]).detach().cpu().numpy()[0])

100%|██████████| 19180/19180 [02:58<00:00, 107.20it/s]


In [24]:
build_faiss_index(
    embeddings=embeddings,
    save_path="/home/omnious/workspace/jovis/jovis-model/outputs/skb",
    save_name="descriptions_v2",
    pids=pids
)